In [ ]:
import cv2
from ultralytics import YOLO
import math
from pymycobot.mycobot import MyCobot
import time
# YOLO 모델 로드
model = YOLO('/path/to/yolo11n-obb.pt')  # 모델 파일 경로 설정
# 카메라 연결 (0은 기본 카메라)
cap = cv2.VideoCapture(0)
# 카메라가 열리지 않으면 종료
if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()
# MyCobot 연결
mc = MyCobot('COM9', 115200)
mc.send_angles([0, 0, 0, 0, 0, 0], 20)
time.sleep(2)
while True:
    try:
        # 프레임 캡처
        ret, frame = cap.read()
        if not ret:
            print("Error: Failed to capture image.")
            break
        # YOLO 모델 실행 (예측)
        result = model(frame)[0]
        # 예측된 OBB(Orientation Box) 좌표 확인
        if result.obb is not None and len(result.obb.xywhr) > 0:
            # 첫 번째 물체만 처리
            box, cls = result.obb.xywhr[0], result.names[0]
            x_center, y_center, width, height, angle_radians = box
            # 라디안을 각도로 변환
            angle_degrees = angle_radians * (180 / math.pi)
            # 각도를 정수로 변환
            angle_degrees_int = int(angle_degrees)
            # 0 ~ 180을 -90 ~ 90으로 변환
            angle_mycobot = angle_degrees_int - 90
            # 올바른 타입으로 변환 (float로)
            x_center, y_center, width, height = map(float, (x_center, y_center, width, height))
            # 회전된 사각형 그리기
            box_points = cv2.boxPoints(((x_center, y_center), (width, height), angle_degrees_int))
            box_points = box_points.astype(int)
            # 회전된 사각형 그리기
            cv2.polylines(frame, [box_points], isClosed=True, color=(0, 255, 0), thickness=2)
            # 클래스 이름을 화면에 표시
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(frame, cls, (int(x_center), int(y_center) - 10), font, 0.9, (255, 255, 0), 2, cv2.LINE_AA)
            # 각도 값을 화면에 표시
            angle_text = f"Angle: {angle_degrees_int}°"  # 각도 정수로 표시
            cv2.putText(frame, angle_text, (int(x_center), int(y_center) + 20), font, 0.9, (255, 0, 0), 2, cv2.LINE_AA)
            # 첫 번째 물체의 각도를 로봇에 전달
            mc.send_angles([0, 0, 0, 0, 0, angle_mycobot], 20)
        else:
            print("No object detected in the frame.")
        # 결과 화면에 표시
        cv2.imshow("YOLO Object Detection (First Object with Angle)", frame)
        # 'q' 키를 눌러 종료
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    except Exception as e:
        print(f"Error occurred: {e}")
        break
# 카메라 리소스 해제
cap.release()
cv2.destroyAllWindows()